# Citibike Data Visualization (03-2020 to 04-2020)

The very first step is to extract data from the website. The simplest way is downloading specific files from the website. Here is another way that I found to be very useful if we want to extract multiple files.

In [ ]:
#import necessary dependencies
import requests
from bs4 import BeautifulSoup as bs
import zipfile
import numpy as np
import pandas as pd
import os, glob

In [ ]:
#url to be queried
url = 'https://s3.amazonaws.com/tripdata/'

#get url content into soup
r = requests.get(url)
soup = bs(r.text, "xml")

#check for the tag
#print(soup.prettify())

In [ ]:
#extract data from soup
data = soup.find_all('Key')

#store text of the data as a list
clean_data = []
for i in range(0,len(data)-1):
    clean_data.append(data[i].text)

#clean_data

In [ ]:
#unzip files from the clean_data list
for cd in clean_data:
    cd_url = url + cd
    
    #download each data file using the new url
    with open(cd, "wb") as f:
        response = requests.get(cd_url)
        f.write(response.content)
    
    #unzip data files
    with zipfile.ZipFile(cd, "r") as zip_ref:
        zip_ref.extractall("tripdata")
        
    #remove the old zipped file after unziping
    os.remove(cd)

With the above script I was able to go over the data at different time period and work on the ones that I found to be interesting. 

After going over all the files, I decided to work on March and April of 2020. For this, I used glob module to retrive the files of 2020 only and then to the specific months. Then the files merged and exported as a single file. 

In [ ]:
#set the working directory
path = "tripdata"

In [ ]:
#match csv files by name/pattern
all_files = glob.glob(os.path.join(path, "20*.csv"))
#all_files

In [ ]:
#combine all files in the list
CHUNK_SIZE=5000
output_file = "Resources/202003_and_04_tripdata.csv"

first_one = True
for month in ['03','04']:
    if not first_one:
        skip_row=[0]
    else:
        skip_row=[]
    
    chunk_container = pd.read_csv("tripdata/2020"+month+"-citibike-tripdata.csv", chunksize=CHUNK_SIZE)
    for chunk in chunk_container:
        chunk.to_csv(output_file,mode="a", index=False)
    first_one = False